In [1]:
class Handle(object):
    """ Handles names for loading and saving different models.
    """
    def __init__(self, epochs=None,
                 batch_size=None,
                 filters=None,
                 filter_size=None,
                 model=None,
                 ftype=None,
                 program=None,
                 dataset=None,
                 extra=None):
        self.epochs = epochs
        self.batch_size = batch_size
        self.filters = filters
        self.filter_size = filter_size

        self.model = model
        self.ftype = ftype
        self.program = program
        self.dataset = dataset

        self.filename = str(self).split('/')[-1]

        self.extra = extra

    def __str__(self):
        return '{0}/{1}_{2}_{3}_{4}_{5}.{6}'.format(self.dataset,
                                                    self.filters,
                                                    self.filter_size,
                                                    self.epochs,
                                                    self.batch_size,
                                                    self.model,
                                                    self.ftype)

    def __repr__(self):
        return '{0}/{1}_{2}_{3}_{4}_{5}.{6}'.format(self.dataset,
                                                    self.filters,
                                                    self.filter_size,
                                                    self.epochs,
                                                    self.batch_size,
                                                    self.model,
                                                    self.ftype)

    def __add__(self, other):
        return str(self) + other

    def __radd__(self, other):
        return other + str(self)

    @classmethod
    def from_filename(cls, filename):
        try:
            basename, ftype, __ = filename.split('.')
        except ValueError:
            basename, ftype = filename.split('.')
        dataset = basename.split('/')[-2]

        info = basename.split('/')[-1]

        filters, filter_size, epochs, batch_size = map(int, info.split('_')[:4])

        model = info.split('_')[-1]

        obj = cls(epochs, batch_size, filters, filter_size, dataset=dataset, model=model, ftype=ftype)

        return obj


In [2]:
from __future__ import print_function, division, absolute_import

import os
import sys
sys.path.insert(0, os.path.abspath('..'))
os.chdir('../CoMET')
import evolutron.networks as nets
from evolutron.engine import DeepTrainer

Using Theano backend.
Using gpu device 1: GeForce GTX 1080 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


In [3]:
# Data and model
data_id = 'zinc'
padded = True
mode = 'unsuper'
model = None

# Architecture
n_filters = 100
filter_length = 30
n_convs = 1
n_fc = 1

# Training
num_epochs = 20
batch_size = 50
optimizer = 'nadam'
rate = .1
validate = .2

input_shape = (500, 20)

In [4]:
net_arch = nets.DeepCoDER.from_options(input_shape, n_conv_layers=n_convs, n_fc_layers=n_fc,
                                       n_filters=n_filters, filter_length=filter_length)

In [5]:
handle = Handle(batch_size=batch_size,
        filter_size=filter_length,
        filters=n_filters,
        program='CoMET',
        dataset=data_id,
        extra=None)
handle.model = 'realDeepCoDER1'

In [9]:
handle

zinc/100_30_None_50_realDeepCoDER1.None

In [10]:
import argparse

In [12]:
parser = argparse.ArgumentParser(description='CoMET - Convolutional Motif Embeddings Tool',
                                 argument_default=argparse.SUPPRESS)

parser.add_argument("data_id", help='The protein dataset to be trained on.')

parser.add_argument("filters", type=int, help='Number of filters in the convolutional layers.')

parser.add_argument("filter_size", type=int,
                    help='Size of filters in the first convolutional layer.')

parser.add_argument("--no_pad", action='store_true',
                    help='Toggle to pad protein sequences. Batch size auto-change to 1.')

parser.add_argument("--mode", choices=['transfer', 'unsupervised', 'supervised'], default='unsupervised')

parser.add_argument("--conv", default=1, type=int,
                    help='number of conv layers.')

parser.add_argument("--fc", default=1, type=int,
                    help='number of conv layers.')

parser.add_argument("-e", "--epochs", default=50, type=int,
                    help='number of training epochs to perform (default: 50)')

parser.add_argument("-b", "--batch_size", type=int, default=50,
                    help='Size of minibatch.')

parser.add_argument("--rate", type=float)

parser.add_argument("--model", type=str,
                    help='Continue training the given model. Other architecture options are unused.')

parser.add_argument("--optimizer", default='nadam', choices=['adam', 'nadam', 'rmsprop',
                                                             'sgd', 'adadelta', 'adagrad'])



_StoreAction(option_strings=['--optimizer'], dest='optimizer', nargs=None, const=None, default='nadam', type=None, choices=['adam', 'nadam', 'rmsprop', 'sgd', 'adadelta', 'adagrad'], help=None, metavar=None)

In [19]:
args = parser.parse_args('zinc 15 20'.split())

In [20]:
args

Namespace(batch_size=50, conv=1, data_id='zinc', epochs=50, fc=1, filter_size=20, filters=15, mode='unsupervised', optimizer='nadam')

In [24]:
args.__dict__

{'batch_size': 50,
 'conv': 1,
 'data_id': 'zinc',
 'epochs': 50,
 'fc': 1,
 'filter_size': 20,
 'filters': 15,
 'mode': 'unsupervised',
 'optimizer': 'nadam'}